# Supply Chain Analysis with Memgraph

## Table of contents <a name="toc"></a>
1. [Introduction](#introduction)
2. [Connect to Memgraph](#connect-to-memgraph)
3. [Create the dataset](#create-the-dataset)
4. [Supply Chain Analysis](#supply-chain-analysis)
    - [Acquiring critical hubs in the network with betweenness centrality](#critical-hubs)
    - [Get ingredients provided by the supplier](#get-ingredients)
    - [Pathfinding for necessary ingredients](#pathfinding)
    - [Checking dependencies of the product with ancestors](#ancestors)
    - [Ancestors graph](#ancestors-graph)
    - [Checking possible products for production with descendants](#descendants)
    - [Descendants graph](#descendants-graph)
    - [Getting the order of execution with topological sort](#topological-sort)
5. [Conclusion](#conclusion)

## 1. Introduction<a name="introduction"></a>

In supply chain management, a network of process steps is drawn to minimize product delivery time from production to shipping. Up to this day, optimizations in process steps are mostly carried out by staff members, who can be prone to errors and under-optimized solutions. Moreover, it takes them a reasonable amount of time to design an optimal schedule when they could have been utilized for processes requiring more expertise and knowledge with an automated process scheduling the supply chain.

In this notebook, you'll learn how to start Memgraph, connect to it and run Cypher queries to explore the supply chain and learn more about the power of graphs in that domain. 

## 2. Connect to Memgraph<a name="connect-to-memgraph"></a>

[Memgraph](https://memgraph.com/) is an open-source in-memory graph database built for highly performant and advanced analytical insights. Memgraph is Neo4j Bolt protocol compatible and uses the standardized Cypher query language. 

For a quick start, run the following command in your terminal to start the Memgraph Platform in a Docker container: 

```
docker run -it -p 7687:7687 -p 7444:7444 -p 3000:3000 --name memgraph memgraph/memgraph-platform
```

The above command starts the Memgraph database, MAGE (graph algorithms library) and Memgraph Lab (visual user interface). For additional instructions on setting up and running Memgraph locally, refer to the [Memgraph documentation](https://memgraph.com/docs/memgraph/installation). 

<em>For Memgraph < 2.11, in order for the Neo4j driver to work, you need [modify configuration setting](https://memgraph.com/docs/configuration/configuration-settings) `--bolt-server-name-for-init`. When running Memgraph, set `--bolt-server-name-for-init=Neo4j/5.2.0`. If you use other version of Neo4j driver, make sure to put the appropriate version number.</em>

After the local setup of Memgraph is complete, set the following configuration to connect from the Graph Notebook:

In [ ]:
%%graph_notebook_config
{
  "host": "localhost",
  "port": 7687,
  "ssl": false
}


### Test the connection

In [ ]:
%%oc bolt
MATCH (n)
RETURN count(n);


## 3. Create the dataset<a name="create-the-dataset"></a>

You can query Memgraph via Bolt protocol designed for efficient communication with graph databases. Memgraph supports versions 1.0, 4.0, 4.1, 4.3, 5.2 of the protocol. Specify the `%%oc bolt` option when submitting queries to the Bolt endpoint.

Before we analyze the dataset, we have to import it. The easiest way to do that with the `graph-notebook` is to run `CREATE` Cypher queries. Once you run the code cell below, the Memgraph database will be populated with a supply chain dataset. 

In [ ]:
%%oc bolt
CREATE (sup1:Supplier {id: 1, name: "Supplissimus"})
CREATE (sup2:Supplier {id: 2, name: "Supplionis"})
CREATE (sup3:Supplier {id: 3, name: "MegaSupplies"})
CREATE (sup4:Supplier {id: 4, name: "Supplies4you"})
CREATE (ing1:Ingredient {id: 1, name: "Ingredient 1"})
CREATE (ing2:Ingredient {id: 2, name: "Ingredient 2"})
CREATE (ing3:Ingredient {id: 3, name: "Ingredient 3"})
CREATE (ing4:Ingredient {id: 4, name: "Ingredient 4"})
CREATE (ing5:Ingredient {id: 5, name: "Ingredient 5"})
CREATE (ing6:Ingredient {id: 6, name: "Ingredient 6"})
CREATE (ing7:Ingredient {id: 7, name: "Ingredient 7"})
CREATE (ing8:Ingredient {id: 8, name: "Ingredient 8"})
CREATE (ing9:Ingredient {id: 9, name: "Ingredient 9"})
CREATE (ing10:Ingredient {id: 10, name: "Ingredient 10"})
CREATE (pro1:Product {id: 1, name: "Intermediate product 1"})
CREATE (pro2:Product {id: 2, name: "Intermediate product 2"})
CREATE (pro3:Product {id: 3, name: "Intermediate product 3"})
CREATE (pro4:Product {id: 4, name: "Intermediate product 4"})
CREATE (pro5:Product {id: 5, name: "Intermediate product 5"})
CREATE (pro6:FinalProduct:Product {id: 6, name: "Final product 1"})
CREATE (pro7:FinalProduct:Product {id: 7, name: "Final product 2"})
CREATE (pro8:FinalProduct:Product {id: 8, name: "Final product 3"})
CREATE (shi1:Shipping {id: 1, name: "Shipping point 1"})
CREATE (shi2:Shipping {id: 2, name: "Shipping point 2"})
CREATE (rec1:Recipe {id: 1, name: "Recipe for product 1"})
CREATE (rec2:Recipe {id: 2, name: "Recipe for product 2"})
CREATE (rec3:Recipe {id: 3, name: "Recipe for product 3"})
CREATE (rec4:Recipe {id: 4, name: "Recipe for product 4"})
CREATE (rec5:Recipe {id: 5, name: "Recipe for product 5"})
CREATE (rec6:Recipe {id: 6, name: "Recipe for final product 1"})
CREATE (rec7:Recipe {id: 7, name: "Recipe for final product 2"})
CREATE (rec8:Recipe {id: 8, name: "Recipe for final product 3 - variant 1"})
CREATE (rec9:Recipe {id: 9, name: "Recipe for final product 3 - variant 2"})
CREATE (rec10:Recipe {id: 10, name: "Recipe for final product 3 - variant 3"})
CREATE (sup1)-[:SUPPLIES]->(ing1)
CREATE (sup1)-[:SUPPLIES]->(ing2)
CREATE (sup1)-[:SUPPLIES]->(ing3)
CREATE (sup1)-[:SUPPLIES]->(ing4)
CREATE (sup2)-[:SUPPLIES]->(ing5)
CREATE (sup2)-[:SUPPLIES]->(ing6)
CREATE (sup2)-[:SUPPLIES]->(ing7)
CREATE (sup3)-[:SUPPLIES]->(ing8)
CREATE (sup3)-[:SUPPLIES]->(ing9)
CREATE (sup4)-[:SUPPLIES]->(ing10)
CREATE (pro1)-[:FORMS {quantity: 15}]->(rec6)
CREATE (pro2)-[:FORMS {quantity: 25}]->(rec6)
CREATE (pro2)-[:FORMS {quantity: 65}]->(rec7)
CREATE (pro2)-[:FORMS {quantity: 100}]->(rec9)
CREATE (pro3)-[:FORMS {quantity: 35}]->(rec6)
CREATE (pro3)-[:FORMS {quantity: 120}]->(rec7)
CREATE (pro4)-[:FORMS {quantity: 130}]->(rec7)
CREATE (pro4)-[:FORMS {quantity: 140}]->(rec8)
CREATE (pro5)-[:FORMS {quantity: 85}]->(rec8)
CREATE (ing1)-[:FORMS {quantity: 30}]->(rec1)
CREATE (ing2)-[:FORMS {quantity: 50}]->(rec1)
CREATE (ing2)-[:FORMS {quantity: 100}]->(rec2)
CREATE (ing2)-[:FORMS {quantity: 50}]->(rec10)
CREATE (ing3)-[:FORMS {quantity: 80}]->(rec1)
CREATE (ing3)-[:FORMS {quantity: 200}]->(rec2)
CREATE (ing4)-[:FORMS {quantity: 150}]->(rec2)
CREATE (ing4)-[:FORMS {quantity: 70}]->(rec10)
CREATE (ing5)-[:FORMS {quantity: 10}]->(rec3)
CREATE (ing6)-[:FORMS {quantity: 90}]->(rec3)
CREATE (ing7)-[:FORMS {quantity: 100}]->(rec3)
CREATE (ing8)-[:FORMS {quantity: 200}]->(rec3)
CREATE (ing9)-[:FORMS {quantity: 300}]->(rec4)
CREATE (ing9)-[:FORMS {quantity: 80}]->(rec5)
CREATE (ing10)-[:FORMS {quantity: 120}]->(rec4)
CREATE (ing10)-[:FORMS {quantity: 5}]->(rec5)
CREATE (ing10)-[:FORMS {quantity: 100}]->(rec9)
CREATE (pro6)-[:SHIPS_WITH]->(shi1)
CREATE (pro7)-[:SHIPS_WITH]->(shi1)
CREATE (pro8)-[:SHIPS_WITH]->(shi2)
CREATE (rec1)-[:PRODUCES {quantity: 1}]->(pro1)
CREATE (rec2)-[:PRODUCES {quantity: 1}]->(pro2)
CREATE (rec3)-[:PRODUCES {quantity: 1}]->(pro3)
CREATE (rec4)-[:PRODUCES {quantity: 1}]->(pro4)
CREATE (rec5)-[:PRODUCES {quantity: 1}]->(pro5)
CREATE (rec6)-[:PRODUCES {quantity: 1}]->(pro6)
CREATE (rec7)-[:PRODUCES {quantity: 1}]->(pro7)
CREATE (rec8)-[:PRODUCES {quantity: 1}]->(pro8)
CREATE (rec9)-[:PRODUCES {quantity: 1}]->(pro8)
CREATE (rec10)-[:PRODUCES {quantity: 1}]->(pro8)


To ensure the data is stored in Memgraph, head to `localhost:3000` and check out Memgraph Lab, a visual user interface. You can see node and relationship count there, explore, query and visualize data. Besides that, you can head over to the Graph Schema tab to check if the imported data is appropriately modeled.

<img src="https://public-assets.memgraph.com/graph-notebook/graph-schema-supply-chain.png" alt="drawing" style="width:500px;"/>

Great! The data is imported into Memgraph, and we can start analyzing it!

## 4. Supply Chain Analysis<a name="supply-chain-analysis"></a>

### Acquiring critical hubs in the network with betweenness centrality<a name="critical-hubs"></a>

If, at some point, a critical path of the pipeline fails, it could mean that some products won't get constructed. Some pipeline failures don't affect as many products and don't need much attention fixing (if the priority isn't high). Some, on the other hand, need immediate attention. 

An algorithm like *betweenness centrality* does just that. It detects hubs on the network based on the number of paths that cross a node from all the pairs of nodes in the graph. 

By running the query below, we can see that some Intermediate products, if missing, could result in having all of the final products not produced, which is a massive error in the pipeline, and needs extra care to prevent that from happening (by having some alternative measures, additional monitoring of intermediate product production, etc.).

Memgraph's support of betweenness centrality is done through the **betweenness_centrality_online.set()** method, which also works in streaming examples.

In [ ]:
%%oc bolt
CALL betweenness_centrality.get() YIELD betweenness_centrality, node
SET node.centrality = betweenness_centrality;


In [ ]:
%%oc bolt
MATCH (n:Ingredient)-[r]->(m)
RETURN n, r, m
ORDER BY n.centrality DESC;


### Get ingredients provided by the supplier<a name="get-ingredients"></a>

Since a graph database can be the ultimate source of truth between different data sources, it makes sense if all the information about our suppliers is stored in Memgraph.

From there, we can query, for example, which ingredients are supplied by the supplier *Supplissimus*.

In [ ]:
%%oc bolt
MATCH (s:Supplier {name:"Supplissimus"})-[r:SUPPLIES]->(i:Ingredient)
RETURN i;


### Pathfinding for necessary ingredients<a name="pathfinding"></a>

We have seen a 1-hop query, which is essentially looking for the nearest neighbors in the network.

Memgraph supports graph traversals, e.g., **Breadth-first search (BFS)**. With it, we can see which ingredients are used to form the product with the ID of 6.

In [ ]:
%%oc bolt
MATCH p=(i:Ingredient)-[*BFS]->(f:FinalProduct {id:6})
RETURN p


### Checking dependencies of the product with ancestors<a name="ancestors"></a>

But traversals are not only a part of graph databases, as whole graph algorithms can be exploited on graph storage like Memgraph. 

This query determines what happens before the **:FinalProduct** with the ID 6 gets produced. It is done using the **graph_util.ancestors** procedure captures all the nodes from which a path to the destination node (FinalProduct) exists. 

In [ ]:
%%oc bolt
MATCH (f:FinalProduct {id:6})
CALL graph_util.ancestors(f) YIELD ancestors
UNWIND ancestors AS ancestor
RETURN ancestor;


### Ancestors graph<a name="ancestors-graph"></a>

The previous procedure has yielded us all the precedent nodes, but it only means a little since we don't know how they are connected. 

To connect the nodes, we can use another MAGE extension procedure called **graph_util.connect_nodes**, which will connect the nodes with corresponding relationships between them.

In [ ]:
%%oc bolt
MATCH (f:FinalProduct {id:6})
CALL graph_util.ancestors(f) YIELD ancestors
WITH ancestors + [f] AS nodes
CALL graph_util.connect_nodes(nodes) YIELD connections
UNWIND nodes + connections AS graph
RETURN graph;


### Checking possible products for production with descendants<a name="descendants"></a>

We might look at the pipeline from the other direction. From the supplier's view, we can see how many products or operations in the pipeline are affected by him. In case he is unavailable, this information could be helpful to minimize the risk.

Just as with ancestors, we use the procedure **graph_util.descendants**, which yields all the nodes to which a path exists from the source node (supplier *Supplissimus* in this case).

In [ ]:
%%oc bolt
MATCH (s:Supplier {name: "Supplissimus"})
CALL graph_util.descendants(s) YIELD descendants
UNWIND descendants AS descendant
RETURN descendant;


### Descendants graph<a name="descendants-graph"></a>

We do the same as before and connect the nodes with the **graph_util.connect_nodes** procedure.

In [ ]:
%%oc bolt
MATCH (s:Supplier {name: "Supplissimus"})
CALL nxalg.descendants(s) YIELD descendants
WITH descendants + [s] AS nodes
CALL graph_util.connect_nodes(nodes) YIELD connections
UNWIND nodes + connections AS graph
RETURN graph;


### Getting the order of execution with topological sort<a name="topological-sort"></a>

There are cases when some operations can't start before others finish, which causes problems because it blocks the pipeline until the process or a job with no dependencies or bottlenecks finishes. Then, some jobs are released and resolved of their dependencies, and they can start executing again. 

In graph theory, that's precisely what topological sort does. It sorts the nodes to yield the ones (jobs, operations, or products) that get executed or produced first, followed by those that can start after the previous ones have started.

For sorting the nodes topologically, we will use **graph_util.topological_sort** procedure.

In [ ]:
%%oc bolt
MATCH p=(r:Recipe)-[*bfs]->(f:FinalProduct)
WITH project(p) AS graph
CALL graph_util.topological_sort(graph) YIELD sorted_nodes
UNWIND sorted_nodes AS nodes
RETURN nodes.name;


## 5. Conclusion<a name="conclusion"></a>

Hopefully, you learned about Memgraph, supply chains and how it's intuitive to analyze them with Cypher queries. If you want to understand why graph databases are the future of network resource optimization, head over to [Memgraph's blog post](https://memgraph.com/blog/graphs-databases-are-the-future-for-network-resource-optimization). For any questions regarding this notebook, Cypher, Memgraph or graphs in general, [join our Discord community](https://www.discord.gg/memgraph). 

<h4 align="center"><a href=#toc>⬆️ GO TO TOP ⬆️</a></h3>